In [1]:
# Библиотеки использованные для обработки данных
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.utils import resample
import pickle

In [2]:
# Датафрейм читается из csv файла - stroke_data.csv - взятого с kaggle
df = pd.read_csv('data/stroke_data.csv')

In [3]:
# Удаление столбца id, который тут не нужен
df.drop('id', axis=1, inplace=True)

In [4]:
# Просмотр получившегося датафрейма 
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Поиск NULL значений в каждом из столбцов

In [5]:
# Вывод NULL значений в каждом из столбцов
print(df.isna().sum())

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


- Подсчет кол-ва NULL, вышло порядка 201 значения на весь датасет и все в столбце bmi.

In [6]:
# Для проверки статистического анализа всех атрибутов числового типа(кол-во, среднее, стандартное отклоненние, минимальное значение, все квартили, максимальное значение)
df.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [7]:
# Предоставление типов данных всех столбцов и количество значений NOT NULL
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [8]:
# Дубликатов не было обнаружено
df.duplicated().sum()

0

## Нахождение выбросов, нормирование атрибута bmi и замена нулевых значений

In [9]:
# Нахождение количества выбросов основываясь на межквартильном размахе(IQR) и их нормирование
Q1 = df['bmi'].quantile(0.05)
Q3 = df['bmi'].quantile(0.95)
IQR = Q3 - Q1
df['bmi'] = np.where(
    (df['bmi'] < (Q1 - 1.5 * IQR)) | (df['bmi'] > (Q3 + 1.5 * IQR)),
    (Q1 - 1.5 * IQR) + np.random.uniform(0, 1) * (Q3 + 1.5 * IQR - (Q1 - 1.5 * IQR)),
    df['bmi']
)

scaler = MinMaxScaler()
df['bmi'] = scaler.fit_transform(df[['bmi']])

with open('models/min_max_scaler_bmi.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [10]:
# Процентное соотношение нулевых значений в bmi
df['bmi'].isna().sum() / len(df['bmi']) * 100

3.9334637964774952

- В датафрейме содержится 3.93 % нулевых значений

In [11]:
df_na = df.loc[df['bmi'].isnull()]
n = df_na['stroke'].sum()
g = df['stroke'].sum()
print(n)
print(g)
print(n / g * 100)

40
249
16.06425702811245


- Люди у которых случился инсульт и их bmi равен 0: 40
- Люди у которых случился инсульт и их bmi не равен 0: 249
- Процент людей с инсультом и значениями 0 в отношении ко всему датасету: 16.06425702811245 

In [12]:
# Процент пациентов, которые получили инсульт
df['stroke'].sum() / len(df) * 100

4.87279843444227

- Наш целевой атрибут это stroke, и пациентов, которые получили инсульт в меньшинстве - 249. Что является лишь 4.9 процентами из всех пациентов.

In [13]:
# Анализ того, стоит ли выбрасывать нулевые значения со столбца bmi
df_na = df.loc[df['bmi'].isnull()]
print(df_na['stroke'].sum())
print(df['stroke'].sum())

40
249


- Среди пациентов с нулевыми значениями bmi, 40 из них получили инсульт из всех 249. Поэтому выбросить нулевые значения не можем.

In [14]:
# Замена нулевых значений, вычисляя медиану к столбцу bmi
df['bmi'] = df['bmi'].fillna(df['bmi'].median())

## Нахождение выбросов, нормирование атрибута avg_glucose_level


In [15]:
# Нахождение количества выбросов основываясь на межквартильном размахе(IQR) и их нормирование
Q1 = df['avg_glucose_level'].quantile(0.05)
Q3 = df['avg_glucose_level'].quantile(0.95)
IQR = Q3 - Q1
df['avg_glucose_level'] = np.where(
    (df['avg_glucose_level'] < (Q1 - 1.5 * IQR)) | (df['avg_glucose_level'] > (Q3 + 1.5 * IQR)),
    (Q1 - 1.5 * IQR) + np.random.uniform(0, 1) * (Q3 + 1.5 * IQR - (Q1 - 1.5 * IQR)),
    df['avg_glucose_level']
)

scaler = MinMaxScaler()
df['avg_glucose_level'] = scaler.fit_transform(df[['avg_glucose_level']])

with open('models/min_max_scaler_avg_glucose_level.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## Замена атрибутов с типом данных object в category

In [16]:
le = LabelEncoder()
text_data_features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
l3 = []; l4 = []
print('Label Encoder Transformation')
for i in tqdm(text_data_features):
    df[i] = le.fit_transform(df[i])
    l3.append(list(df[i].unique())); l4.append(list(le.inverse_transform(df[i].unique())))
    print(i,' : ',df[i].unique(),' = ',le.inverse_transform(df[i].unique()))
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

Label Encoder Transformation


100%|██████████| 5/5 [00:00<00:00, 782.87it/s]

gender  :  [1 0 2]  =  ['Male' 'Female' 'Other']
ever_married  :  [1 0]  =  ['Yes' 'No']
work_type  :  [2 3 0 4 1]  =  ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Residence_type  :  [1 0]  =  ['Urban' 'Rural']
smoking_status  :  [1 2 3 0]  =  ['formerly smoked' 'never smoked' 'smokes' 'Unknown']


In [17]:
# Датафрейм после смены типов данных
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,0.801265,0.388479,1,1
1,0,61.0,0,0,1,3,0,0.679023,0.262925,2,1
2,1,80.0,0,1,1,2,0,0.234512,0.327917,2,1
3,0,49.0,0,0,1,2,1,0.536008,0.355982,3,1
4,0,79.0,1,0,1,3,0,0.549349,0.202363,2,1


In [18]:
# Так как у нас сильный перевес людей без инсульта, решено создать подобные данные. Если этого не сделать, у модели будет перевес
df_stroke_1 = df[df['stroke'] == 1]
df_upsample = resample(df_stroke_1, replace=True, n_samples=5000, random_state=123)

In [19]:
# Соединение изначального датафрейма и с только что созданными данными
df = pd.concat([df, df_upsample])

In [20]:
# Представление значений атрибута stroke после создания и объединения подобных данных
df['stroke'].value_counts()

1    5249
0    4861
Name: stroke, dtype: int64

In [21]:
# Сохранение датафрейма в датасет с названием - data/prepared_dataset.csv
df.to_csv('data/prepared_dataset.csv')